In [ ]:
# the main function wgr_deconv_canonhrf_par iterates through each voxel's data, applies event detection, adjusts event onsets, and performs deconvolution. 
## It returns the deconvolved data, event information, estimated HRF, adjustment values, and parameter estimates.

import numpy as np
import scipy as sps

#takes input data and returns deconvolved data, event onset, estimated HRF, adjustment values, parameter estimates

##data: Input data matrix with dimensions (time points x number of voxels)
##thr: Threshold value for detecting events
##event_lag_max: Maximum time from neural event to BOLD event in bins
##TR: Time resolution (time between data points) in seconds

def wgr_deconv_canonhrf_par(data, thr, event_lag_max, TR):
    N, nvar = data.shape
    even_new = wgr_trigger_onset(data, thr)
    p_m = 3
    T = round(30 / TR)
    data_deconv = np.zeros((N, nvar))
    HRF = np.zeros((T, nvar))
    PARA = np.zeros((3, nvar))
    data_deconv, HRF, event, adjust_global, PARA = [], [], [], [], []
    for i in range(nvar):
        data_deconv_i, HRF_i, event_i, adjust_global_i, PARA_i = wgr_adjust_onset(
            data[:, i], even_new[i], event_lag_max, TR, p_m, T, N
        )
        data_deconv.append(data_deconv_i)
        HRF.append(HRF_i)
        event.append(event_i)
        adjust_global.append(adjust_global_i)
        PARA.append(PARA_i)
    return data_deconv, event, HRF, adjust_global, PARA

#performs the adjustment and deconvolution steps for each voxel
def wgr_adjust_onset(dat, even_new, event_lag_max, TR, p_m, T, N):
    kk = 1
    hrf = np.zeros((T, event_lag_max + 1))
    Cov_E = np.zeros(event_lag_max + 1)
    for event_lag in range(event_lag_max + 1):
        RR = even_new - event_lag
        RR = RR[RR > 0]
        design = np.zeros(N)
        design[RR - 1] = 1
        hrf[:, kk - 1], e3, param_kk = Fit_Canonical_HRF(dat, TR, design, T, p_m)
        Cov_E[kk - 1] = np.cov(e3)
        kk += 1
    ind = np.argmin(Cov_E)
    ad_global = ind - 1
    even_new -= ad_global
    even_new = even_new[even_new > 0]
    hrf = hrf[:, ind - 1]
    param = param_kk[:, ind - 1]
    H = np.fft.fft(np.vstack((hrf, np.zeros((N - T, 1)))), axis=0)
    M = np.fft.fft(dat)
    dat_deconv = np.fft.ifft(
        np.conj(H) * M / (H * np.conj(H) + Cov_E[ind - 1]), axis=0
    ).real
    return dat_deconv, hrf, even_new, ad_global, param

#detects event onsets in the input data matrix and returns a list of event onsets for each voxel
def wgr_trigger_onset(matrix, thr):
    N, nvar = matrix.shape
    matrix = (matrix - np.mean(matrix, axis=0)) / np.std(matrix, axis=0)
    oneset = []
    for i in range(nvar):
        oneset_temp = []
        for t in range(1, N - 1):
            if (
                matrix[t, i] > thr
                and matrix[t - 1, i] < matrix[t, i]
                and matrix[t, i] > matrix[t + 1, i]
            ):
                oneset_temp.append(t)
        oneset.append(oneset_temp)
    return oneset

#fits GLM using the canonical HRF and its derivatives
def Fit_Canonical_HRF(tc, TR, Run, T, p):
    len_Run = len(Run)
    X = np.zeros((len_Run, p))
    h, dh, dh2 = CanonicalBasisSet(TR)
    v = np.convolve(Run, h)[:len_Run]
    X[:, 0] = v
    if p > 1:
        v = np.convolve(Run, dh)[:len_Run]
        X[:, 1] = v
    if p > 2:
        v = np.convolve(Run, dh2)[:len_Run]
        X[:, 2] = v
    X = np.column_stack((np.zeros(len_Run) + 1, X))
    b = np.linalg.pinv(X) @ tc
    e = tc - X @ b
    fit = X @ b
    b = b[1:]
    if p == 2:
        bc = np.sign(b[0]) * np.sqrt(b[0] ** 2 + b[1] ** 2)
        H = np.column_stack((h, dh))
    elif p == 1:
        bc = b[0]
        H = h
    elif p > 2:
        bc = np.sign(b[0]) * np.sqrt(b[0] ** 2 + b[1] ** 2 + b[2] ** 2)
        H = np.column_stack((h, dh, dh2))
    hrf = H @ b
    param = get_parameters2(hrf, T)
    return hrf, e, param

#generates canonical HRF and derivatives
def CanonicalBasisSet(TR):
    len_h = round(30 / TR)
    v1 = np.ones(len_h)
    v2 = v1 - np.convolve(v1, np.array([1, -1]), "valid")
    v3 = v2 - np.convolve(v2, np.array([1, -1]), "valid")
    h = v1 / np.max(v1)
    dh = v2 / np.max(v2)
    dh2 = v3 / np.max(v3)
    return h, dh, dh2

# calculates model parameters (height, time to peak, and width) for the estimated HRF
def get_parameters2(hdrf, t):
    n = int(round(t * 0.8))
    h, p = np.max(np.abs(hdrf[:n])), np.argmax(np.abs(hdrf[:n])) + 1
    if h > 0:
        v = hdrf >= h / 2
    else:
        v = hdrf <= h / 2
    b = np.diff(v.astype(int))
    w = np.sum(b > 0)
    cnt = p - 2
    g = hdrf[1:] - hdrf[:-1]
    while cnt >= 0 and np.abs(g[cnt]) < 0.001:
        h = hdrf[cnt]
        p = cnt + 1
        cnt -= 1
    param = np.array([h, p, w])
    return param
